# Fitbit Foodlog + Activity
## From Takeout
### Setup the basics

In [ ]:
# Install dependancies
# sudo apt-get install texlive-xetex texlive-fonts-recommended texlive-plain-generic
! pip install pandas matplotlib nbconvert PyPDF2

In [52]:

# Importing required libraries
import os, json
import pandas as pd
from IPython.display import display, HTML
import matplotlib.pyplot as plt

# Global Options
pd.set_option('display.max_rows', None)

Import the takeout data and generate pandas dataframes for each set of data

In [ ]:
# Importing the Takeout data
root_folder = './dataset/takeout/Takeout/Fitbit'
dataframes_list = {}

for relpath, dirs, files in os.walk(root_folder):
  folder_name = os.path.basename(relpath)
  dataframes = []
  for file in files:
    custom_name = None
    file_path = os.path.join(relpath, file)
    df = None
    name = None
    # Handle files in Global Export Data seperately
    if folder_name == "Global Export Data":
      if file.startswith('food_logs'):
        df = pd.read_json(file_path)
        name = "Food Log"
      elif file.startswith('water_logs'):
        df = pd.read_json(file_path)
        name = "Water Log"
      elif file.startswith('weight'):
        df = pd.read_json(file_path)
        name = "Weight"
      else:
        continue
    # Import CSV/JSON from other folders
    else:
      if file.endswith('.csv'):
        df = pd.read_csv(file_path)
        name = folder_name
      elif file.endswith('.json'):
        df = pd.read_json(file_path)
        name = folder_name
    # # Add non-empty dataframes to the output
    if df is not None and not df.empty and not df.isna().all().all():
      if name in dataframes_list:
        dataframes_list[name].append(df)
      else:
        dataframes_list[name] = [df]

# Concatenate DataFrames
for name, dfs in dataframes_list.items():
  if dfs:
    concatenated_df = pd.concat(dfs, ignore_index=True)
    dataframes_list[name] 

# Function for displaying dataframe contents   
def get_log(name):
  df_list = dataframes_list.get(name)
  df = pd.concat(df_list, ignore_index=True)
  return df

# Function for displaying a bar graph
def bar_graph(df,xaxis,yaxis,title,figsize):
  df = df.groupby(xaxis, as_index=False).agg({yaxis: 'sum'})
  plt.bar(df[xaxis], df[yaxis])
  plt.title(title)
  plt.xlabel(xaxis)
  plt.ylabel(yaxis)
  plt.xticks(rotation=80)
  plt.grid(True)
  return plt

In [ ]:
for name, df in dataframes_list.items():
    print(f"DataFrame: {name}")

### Display Food Log

In [ ]:
fl = get_log('Food Log')
# Parse the food log JSON
fl_logged = pd.json_normalize(fl['loggedFood'])
# Combine with original dataframe
fl = fl.drop(columns=['loggedFood']).join(fl_logged)
# Display only usable data
# Mapping dictionary
meal_type_mapping = {
    1: 'Breakfast',
    2: 'Morning Snack',
    3: 'Lunch',
    4: 'Afternoon Snack',
    5: 'Dinner',
    6: 'Evening Snack'
}
fl['mealType'] = fl['mealTypeId'].map(meal_type_mapping)
disfl = fl[['logDate','brand','name','calories','amount','unit.name','mealType']]
disfl.columns = ['Date','Brand','Name','Calories','Amount','Units','Meal']

# Display as a nice HTML table
display(HTML(disfl.to_html(index=False)))

### Display Water Log (Raw Data)

In [ ]:
wl = get_log('Water Log')
diswl = wl[['date','waterAmount','measurementUnit']]
diswl = wl.groupby('date', as_index=False).agg({
  'waterAmount': 'sum',
  'measurementUnit': 'first'
  })
diswl.columns = ['Date','Water Amount','Units']
display(diswl.style.hide(axis='index'))

### Calories per day

In [ ]:
# Plotting the graph
bar_graph(disfl, 'Date','Calories','Calorie Intake per Day',(10,6))

# Display the plot
plt.show()
